# Monte Carlo Model Options Pricing Simulation Engine

### Initialized imports and variables

In [30]:
try:
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    from dataclasses import dataclass
    import yfinance as yf
except Exception as exc:
    raise ImportError("Failed to import required libraries. Please check your environment.") from exc

# seed for reproducibility in examples
rng = np.random.default_rng(0)

@dataclass(frozen=True)
class Market:
    S: float  # current stock price
    T: float  # time to maturity in years
    r: float  # risk free interest rate
    q: float  # dividend yield
    sigma: float  # volatility

@dataclass(frozen=True)
class Contract:
    K: float  # strike price
    kind: str  # e.g. "euro_call", "euro_put", "digital_call", "asian_call", "up_and_out_call"
    B: float | None = None  # barrier if needed
    Q: float = 1.0  # digital payout if needed


### Geometric Paths Function + Plot

In [31]:
def monte_carlo_sim_paths(market, steps, trial_count, rng):
    """Simulate full GBM paths.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        steps (int): number of time steps
        trial_count (int): number of paths
        rng (np.random.Generator): RNG for draws

    Returns:
        np.ndarray: shape (steps + 1, trial_count), including S0
    """

    S = market.S
    T = market.T
    r = market.r
    q = market.q
    sigma = market.sigma

    dt = T / steps
    mu = r - q  # expected return

    Z = rng.normal(size=(steps, trial_count))
    lnST = np.log(S) + np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z, axis=0)
    ST = np.exp(lnST)
    S0 = np.full((1, trial_count), S)
    paths = np.vstack([S0, ST])

    return paths

def monte_carlo_sim_terminal(market, trial_count, rng):
    """Simulate terminal prices only.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        trial_count (int): number of paths
        rng (np.random.Generator): RNG for draws

    Returns:
        np.ndarray: shape (trial_count,)
    """

    S = market.S
    T = market.T
    r = market.r
    q = market.q
    sigma = market.sigma

    Z = rng.normal(size=trial_count)
    mu = r - q  # expected return
    drift = (mu - 0.5 * sigma**2) * T
    diff = sigma * np.sqrt(T) * Z

    return S * np.exp(drift + diff)

def plot_paths(market, steps, trial_count, rng):
    """Plot a handful of simulated paths.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        steps (int): number of time steps
        trial_count (int): number of paths to plot
        rng (np.random.Generator): RNG for draws

    Returns:
        None
    """

    paths = monte_carlo_sim_paths(market, steps, trial_count, rng)
    plt.plot(paths)
    plt.xlabel("Time Increments")
    plt.ylabel("Stock Price")
    plt.title("Example Paths")

### Payoff Library (Vectorized)

In [32]:
def payoff_european_call(ST, K):
    return np.maximum(ST - K, 0.0)

def payoff_european_put(ST, K):
    return np.maximum(K - ST, 0.0)

def payoff_digital_call(ST, K, Q=1.0):
    return Q * (ST > K).astype(float)

def payoff_asian_arithmetic_call(S_path, K):
    avg = S_path[1:].mean(axis=0)
    return np.maximum(avg - K, 0.0)

def payoff_up_and_out_call(S_path, K, B):
    knocked_out = S_path.max(axis=0) >= B
    return np.where(knocked_out, 0.0, np.maximum(S_path[-1] - K, 0.0))

### Black–Scholes formula (for validity checks)

In [33]:
from scipy.stats import norm

def black_scholes_test(market, contract):
    """Black–Scholes price for euro_call/euro_put.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        contract (Contract): kind must be euro_call or euro_put

    Returns:
        float: option price
    """

    S = market.S
    T = market.T
    K = contract.K
    r = market.r
    q = market.q
    sigma = market.sigma

    if T <= 0:
        if contract.kind == "euro_call":
            return max(S - K, 0.0)
        elif contract.kind == "euro_put":
            return max(K - S, 0.0)
        else:
            raise ValueError("Black-Scholes only supports euro_call/euro_put")

    vol_sqrt = sigma * np.sqrt(T)
    mu = r - q
    d1 = (np.log(S / K) + (mu + 0.5 * sigma**2) * T) / vol_sqrt
    d2 = d1 - vol_sqrt

    if contract.kind == "euro_call":
        return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    if contract.kind == "euro_put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

    raise ValueError("Black-Scholes only supports euro_call/euro_put")

def test_european_call_mc_vs_bs(market, contract, trial_count, rng):
    """MC vs Black–Scholes check for a euro_call (prints stats).

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        contract (Contract): must be euro_call
        trial_count (int): number of trials
        rng (np.random.Generator): RNG for draws

    Returns:
        tuple: (mc_price, (ci_low, ci_high), bs_price, stderr)
    """

    S = market.S
    T = market.T
    K = contract.K
    r = market.r
    q = market.q
    sigma = market.sigma

    # 1) Simulate terminal prices (expect ST shape: (trial_count,))
    ST = monte_carlo_sim_terminal(market, trial_count, rng)

    # 2) Payoffs
    payoffs = payoff_european_call(ST, K)

    # 3) Discounted payoff
    disc_payoffs = np.exp(-r * T) * payoffs

    # 4) MC estimate + standard error + 95% CI
    mc_price = disc_payoffs.mean()
    stderr = disc_payoffs.std(ddof=1) / np.sqrt(trial_count)
    ci_low, ci_high = mc_price - 1.96 * stderr, mc_price + 1.96 * stderr

    # 5) Black–Scholes
    bs_call = black_scholes_test(market, contract)

    abs_err = mc_price - bs_call
    se_err = abs_err / stderr if stderr > 0 else np.nan

    print(f"MC Call Price: {mc_price:.6f}")
    print(f"MC 95% CI    : [{ci_low:.6f}, {ci_high:.6f}]  (stderr={stderr:.6f})")
    print(f"BS Call Price: {bs_call:.6f}")
    print(f"Error (MC-BS): {abs_err:.6f}  ({se_err:.2f} SE)")

    return mc_price, (ci_low, ci_high), bs_call, stderr


### Core Engine

In [34]:
def price_mc(
    market,
    contract,
    steps=200,
    trial_count=100_000,
    batch_size=50_000,
    antithetic=True,
    rng=None,
):
    """Monte Carlo pricer with batching and optional antithetic draws.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        contract (Contract): option to price
        steps (int): time steps for path-dependent payoffs
        trial_count (int): total number of paths
        batch_size (int): paths per batch
        antithetic (bool): use antithetic variates for terminal payoffs
        rng (np.random.Generator | None): RNG for draws

    Returns:
        tuple: (price, stderr, (ci_low, ci_high))
    """
    if rng is None:
        rng = np.random.default_rng()
    if trial_count <= 0:
        raise ValueError("trial_count must be positive")
    if batch_size <= 0:
        raise ValueError("batch_size must be positive")

    T = market.T
    r = market.r
    kind = contract.kind
    discount = np.exp(-r * T)

    def terminal_prices(n):
        if not antithetic or n == 1:
            return monte_carlo_sim_terminal(market, n, rng)

        half = n // 2
        S = market.S
        q = market.q
        sigma = market.sigma
        mu = r - q
        drift = (mu - 0.5 * sigma**2) * T
        diff = sigma * np.sqrt(T) * rng.normal(size=half)

        # paired shocks for antithetic variates
        ST = S * np.exp(drift + diff)
        ST_anti = S * np.exp(drift - diff)
        out = np.concatenate([ST, ST_anti])

        if n % 2 == 1:
            extra = monte_carlo_sim_terminal(market, 1, rng)
            out = np.concatenate([out, extra])

        return out

    total = 0  # number of payoffs seen so far
    sum_payoffs = 0.0
    sum_sq = 0.0

    while total < trial_count:
        n = min(batch_size, trial_count - total)

        if kind in ("euro_call", "euro_put", "digital_call"):
            ST = terminal_prices(n)
            if kind == "euro_call":
                payoffs = payoff_european_call(ST, contract.K)
            elif kind == "euro_put":
                payoffs = payoff_european_put(ST, contract.K)
            else:
                payoffs = payoff_digital_call(ST, contract.K, contract.Q)

        elif kind == "asian_call":
            paths = monte_carlo_sim_paths(market, steps, n, rng)
            payoffs = payoff_asian_arithmetic_call(paths, contract.K)

        elif kind == "up_and_out_call":
            if contract.B is None:
                raise ValueError("Barrier B required for up_and_out_call")
            paths = monte_carlo_sim_paths(market, steps, n, rng)
            payoffs = payoff_up_and_out_call(paths, contract.K, contract.B)

        else:
            raise ValueError(f"Unsupported contract kind: {kind}")

        disc = discount * payoffs
        sum_payoffs += disc.sum()
        sum_sq += np.square(disc).sum()
        total += n

    price = sum_payoffs / trial_count
    if trial_count > 1:
        var = (sum_sq - trial_count * price**2) / (trial_count - 1)
        if var < 0:
            var = 0.0  # guard tiny negative variance from roundoff
        stderr = np.sqrt(var / trial_count)
    else:
        stderr = 0.0

    ci = (price - 1.96 * stderr, price + 1.96 * stderr)
    return price, stderr, ci



def print_price(
    market,
    contract,
    steps=200,
    trial_count=100_000,
    batch_size=50_000,
    antithetic=True,
    rng=None,
):
    """Print a formatted price line and return the numeric results.

    Inputs:
        market (Market): model inputs (S, T, r, q, sigma)
        contract (Contract): option to price
        steps (int): time steps for path-dependent payoffs
        trial_count (int): total number of paths
        batch_size (int): paths per batch
        antithetic (bool): use antithetic variates for terminal payoffs
        rng (np.random.Generator | None): RNG for draws

    Returns:
        tuple: (price, stderr, (ci_low, ci_high))
    """
    price, stderr, (ci_low, ci_high) = price_mc(
        market,
        contract,
        steps=steps,
        trial_count=trial_count,
        batch_size=batch_size,
        antithetic=antithetic,
        rng=rng,
    )

    def fmt(x):
        return f"{x:,.6f}"

    print(
        f"{contract.kind:<15} price={fmt(price)} | 95% CI [{fmt(ci_low)}, {fmt(ci_high)}] | stderr={fmt(stderr)}"
    )
    return price, stderr, (ci_low, ci_high)


def market_from_yfinance(ticker, T, r, lookback_days=252, interval="1d"):
    """Build a Market from yfinance (spot, div yield, hist vol).

    Inputs:
        ticker (str): symbol, e.g. "AAPL"
        T (float): time to maturity in years
        r (float): risk-free rate (annualized)
        lookback_days (int): window for historical vol
        interval (str): yfinance interval (e.g. "1d")

    Returns:
        Market: filled with S, T, r, q, sigma
    """

    # pull enough history for vol estimation
    hist = yf.download(ticker, period="2y", interval=interval, progress=False)
    if hist.empty:
        raise ValueError(f"No data for ticker: {ticker}")

    close = hist["Close"]
    if isinstance(close, pd.DataFrame):
        close = close[ticker] if ticker in close.columns else close.iloc[:, 0]
    close = close.dropna()
    if close.empty:
        raise ValueError(f"No close prices for ticker: {ticker}")

    # use most recent close as spot
    S = float(close.iloc[-1])

    # historical volatility from log returns
    lookback = close.iloc[-lookback_days:]
    rets = np.log(lookback / lookback.shift(1)).dropna()
    sigma = float(rets.std(ddof=1) * np.sqrt(252))

    info = yf.Ticker(ticker).info
    q = float(info.get("dividendYield") or 0.0)

    return Market(S=S, T=T, r=r, q=q, sigma=sigma)


### Example Usage

In [35]:
# synthetic example
S = 100
K = 105
T = 0.5
r = 0.04
q = 0.01
sigma = 0.25

mkt = Market(S, T, r, q, sigma)

bs_call = black_scholes_test(mkt, Contract(K, "euro_call"))
bs_put = black_scholes_test(mkt, Contract(K, "euro_put"))

_ = print_price(mkt, Contract(K, "euro_call"), trial_count=200_000, rng=rng)
print(f"BS call: {bs_call:.6f}")
print()

_ = print_price(mkt, Contract(K, "euro_put"), trial_count=200_000, rng=rng)
print(f"BS put : {bs_put:.6f}")
print()

_ = print_price(mkt, Contract(K, "asian_call"), steps=200, trial_count=100_000, rng=rng)
_ = print_price(mkt, Contract(K, "up_and_out_call", B=130), steps=200, trial_count=100_000, rng=rng)


euro_call       price=5.546790 | 95% CI [5.501685, 5.591895] | stderr=0.023013
BS call: 5.548166

euro_put        price=8.982628 | 95% CI [8.936769, 9.028488] | stderr=0.023398
BS put : 8.967779

asian_call      price=2.386285 | 95% CI [2.355525, 2.417045] | stderr=0.015694
up_and_out_call price=2.243013 | 95% CI [2.213212, 2.272814] | stderr=0.015205


In [36]:
# real data example (requires internet)
real_mkt = market_from_yfinance("AAPL", T=0.5, r=0.04)
strike = real_mkt.S * 1.05  # 5% OTM
_ = print_price(real_mkt, Contract(strike, "euro_call"), trial_count=100_000, rng=rng)
bs_call_real = black_scholes_test(real_mkt, Contract(strike, "euro_call"))
print(f"BS call: {bs_call_real:.6f}")

euro_call       price=4.126277 | 95% CI [4.030953, 4.221601] | stderr=0.048635
BS call: 4.185298
